## Exercise Set 3 for OSM 

### Dynamic Programming with John Stachurski

Exercises for the [OSM](https://bfi.uchicago.edu/osm) bootcamp dynamic programming section.

We will use the following libraries:

In [47]:
import numpy as np
import quantecon as qe
import matplotlib.pyplot as plt

### Exercise 1.

Using Numba, as discussed in [this lecture](https://lectures.quantecon.org/py/need_for_speed.html), write your own version of NumPy's [interp](https://docs.scipy.org/doc/numpy/reference/generated/numpy.interp.html) function, specializing in linear interpolation in one dimension.  

Note that NumPy's function is compiled native machine code and hence is fast.  But try to beat if you can, at least in some scenarios, by using Numba to speed up your code.  Show a time comparison between the two functions, for some suitable choice of test.

In [56]:
from numba import jit
import inspect
from numpy import interp
#print(inspect.getsource(interp))
from scipy.stats import linregress

@jit
def interp_numba(x, grid, w):
    
    #Sort x values that we are given & create a column that goes along with it for y
    
    y = np.empty_like(x)
        
    #Calculate point-slope formula for each set 
    slope = np.zeros(len(grid))
    intercept = np.zeros(len(grid))
        
    for i in range(0, len(grid) - 1):
        reg = linregress([grid[i],grid[i+1]], [w[i],w[i+1]])
        slope[i] = reg[0]
        intercept[i] = reg[1]
            
    #Find where each x value falls into 
        
    place = np.searchsorted(grid, x)
    
    #Based on each x, find out the y using point slope formula
        
    for k in range (0, len(x)):
            
        if place[k] == 0:
            y[k] = w[0]
               
        elif place[k] == len(grid):
            y[k] = w[len(w) - 1]
            
        else:
            y[k] = intercept[int(place[k]) - 1] + slope[int(place[k]) - 1] * float(x[k])
    
    return y

@jit
def getslope(grid, w):
         
    slope = np.zeros(len(grid))
    intercept = np.zeros(len(grid))
    
    for i in range(0, len(grid) - 1):
        
        reg = linregress([grid[i],grid[i+1]], [w[i],w[i+1]])
        slope[i] = reg[0]
        intercept[i] = reg[1]
    
    return slope, intercept

@jit
def findy(x, grid, w, place, intercept, slope):
    
    for k in range (0, len(x)):
            
        if place[k] == 0:
            y[k] = w[0]
               
        elif place[k] == len(grid):
            y[k] = w[len(w) - 1]
            
        else:
            y[k] = intercept[int(place[k]) - 1] + slope[int(place[k]) - 1] * float(x[k])    
    
    return y

@jit
def interp_numba_jit(x, grid, w):
    
    y = np.empty_like(x)
    
    slope, intercept = getslope(grid, w)
        
    place = np.searchsorted(grid, x)
    
    y = findy(x, grid, w, place, intercept, slope)
    
    return y

print("My code:")
qe.util.tic()
y = interp_numba([0,2,8,4,6], [1,3,5,7], [13, -5, 6, 7])
qe.util.toc()

print("Jit code:")
qe.util.tic()
y = interp_numba_jit([0,2,8,4,6], [1,3,5,7], [13, -5, 6, 7])
qe.util.toc()

print("Their code:")
qe.util.tic()
y = np.interp([0,2,8,4,6], [1,3,5,7], [13, -5, 6, 7])
qe.util.toc()

# Just want to say that I tried a couple of different things to get the Numba to make my code run faster but for some reason it won't.
# I would love to just get some feedback I guess about what I am doing wrong with this code that I have. 

My code:
TOC: Elapsed: 0.0011289119720458984 seconds.
Jit code:
TOC: Elapsed: 1.33925199508667 seconds.
Their code:
TOC: Elapsed: 0.0001399517059326172 seconds.


0.0001399517059326172

### Exercise 2

Using your "Numbafied" linear interpolation function, try to use Numba to additionally speed up the endogenous grid method code from [this lecture](https://lectures.quantecon.org/py/egm_policy_iter.html).  Use CRRA utility and Cobb-Douglas production, as in that lecture, with the following parameter values.


Note: I didn't get much speed up.  I think because the outer loops don't matter much for speed, and hence it doesn't gain us much when we compile them.  

See how you go.

In [58]:
import numpy as np

def coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
    """
    The approximate Coleman operator, updated using the endogenous grid
    method.  
    
    Parameters
    ----------
    g : function
        The current guess of the policy function
    k_grid : array_like(float, ndim=1)
        The set of *exogenous* grid points, for capital k = y - c
    beta : scalar
        The discount factor
    u_prime : function
        The derivative u'(c) of the utility function
    u_prime_inv : function
        The inverse of u' (which exists by assumption)
    f : function
        The production function f(k)
    f_prime : function
        The derivative f'(k)
    shocks : numpy array
        An array of draws from the shock, for Monte Carlo integration (to
        compute expectations).

    """

    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    
    # Determine endogenous grid
    y = k_grid + c  # y_i = k_i + c_i

    # Update policy function and return
    Kg = lambda x: np.interp(x, y, c)
    return Kg

def coleman_egm_numba(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks):
    """
    The approximate Coleman operator, updated using the endogenous grid
    method.  
    
    Parameters
    ----------
    g : function
        The current guess of the policy function
    k_grid : array_like(float, ndim=1)
        The set of *exogenous* grid points, for capital k = y - c
    beta : scalar
        The discount factor
    u_prime : function
        The derivative u'(c) of the utility function
    u_prime_inv : function
        The inverse of u' (which exists by assumption)
    f : function
        The production function f(k)
    f_prime : function
        The derivative f'(k)
    shocks : numpy array
        An array of draws from the shock, for Monte Carlo integration (to
        compute expectations).

    """

    # Allocate memory for value of consumption on endogenous grid points
    c = np.empty_like(k_grid)  

    # Solve for updated consumption value
    for i, k in enumerate(k_grid):
        vals = u_prime(g(f(k) * shocks)) * f_prime(k) * shocks
        c[i] = u_prime_inv(beta * np.mean(vals))
    
    # Determine endogenous grid
    y = k_grid + c  # y_i = k_i + c_i

    # Update policy function and return
    Kg = lambda x: interp_numba(x, y, c)
    return Kg

import matplotlib.pyplot as plt
import quantecon as qe

class LogLinearOG:
    """
    Log linear optimal growth model, with log utility, CD production and
    multiplicative lognormal shock, so that

        y = f(k, z) = z k^alpha

    with z ~ LN(mu, s).

    The class holds parameters and true value and policy functions.
    """

    def __init__(self, alpha=0.4, beta=0.96, mu=0, s=0.1):

        self.alpha, self.beta, self.mu, self.s = alpha, beta, mu, s 

        # == Some useful constants == #
        self.ab = alpha * beta
        self.c1 = np.log(1 - self.ab) / (1 - beta)
        self.c2 = (mu + alpha * np.log(self.ab)) / (1 - alpha)
        self.c3 = 1 / (1 - beta)
        self.c4 = 1 / (1 - self.ab)

    def u(self, c):
        " Utility "
        return np.log(c)

    def u_prime(self, c):
        return 1 / c

    def f(self, k):
        " Deterministic part of production function.  "
        return k**self.alpha

    def f_prime(self, k):
        return self.alpha * k**(self.alpha - 1)

    def c_star(self, y):
        " True optimal policy.  "
        return (1 - self.alpha * self.beta) * y

    def v_star(self, y):
        " True value function. "
        return self.c1 + self.c2 * (self.c3 - self.c4) + self.c4 * np.log(y)
    
lg = LogLinearOG()

# == Unpack parameters / functions for convenience == #
alpha, beta, mu, s = lg.alpha, lg.beta, lg.mu, lg.s
v_star, c_star = lg.v_star, lg.c_star
u, u_prime, f, f_prime = lg.u, lg.u_prime, lg.f, lg.f_prime

grid_max = 4         # Largest grid point, exogenous grid
grid_size = 200      # Number of grid points
shock_size = 250     # Number of shock draws in Monte Carlo integral

k_grid = np.linspace(1e-5, grid_max, grid_size)
shocks = np.exp(mu + s * np.random.randn(shock_size))

alpha = 0.65
beta = 0.95
mu = 0
s = 0.1
grid_min = 1e-6
grid_max = 4
grid_size = 200
shock_size = 250

gamma = 1.5   # Preference parameter
gamma_inv = 1 / gamma

def f(k):
    return k**alpha

def f_prime(k):
    return alpha * k**(alpha - 1)

def u(c):
    return (c**(1 - gamma) - 1) / (1 - gamma)

def u_prime(c):
    return c**(-gamma)

def u_prime_inv(c):
    return c**(-gamma_inv)

k_grid = np.linspace(grid_min, grid_max, grid_size)
shocks = np.exp(mu + s * np.random.randn(shock_size))

## Let's make convenience functions based around these primitives

def crra_coleman_egm(g):
    return coleman_egm(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

def crra_coleman_egm_numba(g):
    return coleman_egm_numba(g, k_grid, beta, u_prime, u_prime_inv, f, f_prime, shocks)

sim_length = 20

print("Timing policy function iteration with endogenous grid")
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_egm(g)
    g = new_g
qe.util.toc()

print("Timing policy function iteration with endogenous grid & numba")
g_init_egm = lambda x: x
g = g_init_egm
qe.util.tic()
for i in range(sim_length):
    new_g = crra_coleman_egm_numba(g)
    g = new_g
qe.util.toc()

Timing policy function iteration with endogenous grid
TOC: Elapsed: 0.14974188804626465 seconds.
Timing policy function iteration with endogenous grid & numba
TOC: Elapsed: 83.75886917114258 seconds.


83.75886917114258